# A2 - Bias in Data
### DATA 512
### Laura Thriftwood

The purpose...

In [1]:
import pandas as pd
import numpy as np
import json
import math
import requests

## Step 1: Getting the Article and Population Data

The Wikipedia politicians by country dataset comes from Figshare. The .zip file was downloaded and unzipped where the _page_data.csv_ file was located.

The population data was drawn from the World Population data sheet published by the Population Reference Bureau and was downloaded as a .csv file named _WPDS_2020_data.csv_. 

These files can be located in the __data__ folder.


## Step 2: Cleaning the Data

First let's look at the politicians by country dataset


In [2]:
df_politicians = pd.read_csv('data/country/data/page_data.csv')

In [3]:
df_politicians.head()

,page,country,rev_id
0,Template:ZambiaProvincialMinisters,Zambia,235107991
1,Bir I of Kanem,Chad,355319463
2,Template:Zimbabwe-politician-stub,Zimbabwe,391862046
3,Template:Uganda-politician-stub,Uganda,391862070
4,Template:Namibia-politician-stub,Namibia,391862409


In [4]:
df_politicians.shape

(47197, 3)

We want to remove/ignore the rows of data that include "Template:" in the string of the page name as these are not Wikipedia articles and should not be included in the analysis.

In [5]:
df_politicians = df_politicians[~df_politicians.page.str.contains('Template:')].reset_index(drop=True)

In [6]:
df_politicians.head()

,page,country,rev_id
0,Bir I of Kanem,Chad,355319463
1,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188
2,Yos Por,Cambodia,393822005
3,Julius Gregr,Czech Republic,395521877
4,Edvard Gregr,Czech Republic,395526568


In [7]:
df_politicians.shape

(46701, 3)

We can see that this removed 496 rows. Now let's take a look at our other dataset, WPDS_2020_data.csv

In [8]:
df_population = pd.read_csv('data/WPDS_2020_data.csv')
df_population.head(30)

,FIPS,Name,Type,TimeFrame,Data (M),Population
0,WORLD,WORLD,World,2019,7772.850,7772850000
1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000
2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000
3,DZ,Algeria,Country,2019,44.357,44357000
4,EG,Egypt,Country,2019,100.803,100803000
5,LY,Libya,Country,2019,6.891,6891000
6,MA,Morocco,Country,2019,35.952,35952000
7,SD,Sudan,Country,2019,43.849,43849000
8,TN,Tunisia,Country,2019,11.896,11896000
9,EH,Western Sahara,Country,2019,0.597,597000


In [9]:
df_population.to_csv(r'before_sort.csv', index = False, header = True)

We want to ignore rows that provide cumulative regional population counts, rather than country-level counts. These rows are distinguished by having ALL CAPS values in the __Name__ field. We can move these into a separate dataframe to reference later when reporting coverage and quality by region in the analysis section.

Note: Initially, I used the designation in the __Type__ field to determine exclusion criteria but there exists an entry for Channel Islands that has a Sub-Region __Type__ but is not displayed in ALL CAPS in the __Name__ field.

As we want to preserve the data we are removing in this step, we first make a copy of the population data.

In [10]:
df_sub_region_population = df_population.copy()
df_sub_region_population = df_sub_region_population[df_sub_region_population['Name'].str.isupper().fillna(False)]
df_sub_region_population

,FIPS,Name,Type,TimeFrame,Data (M),Population
0,WORLD,WORLD,World,2019,7772.850,7772850000
1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000
2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000
10,WESTERN AFRICA,WESTERN AFRICA,Sub-Region,2019,401.115,401115000
27,EASTERN AFRICA,EASTERN AFRICA,Sub-Region,2019,444.970,444970000
48,MIDDLE AFRICA,MIDDLE AFRICA,Sub-Region,2019,179.757,179757000
58,SOUTHERN AFRICA,SOUTHERN AFRICA,Sub-Region,2019,67.732,67732000
64,NORTHERN AMERICA,NORTHERN AMERICA,Sub-Region,2019,368.193,368193000
67,LATIN AMERICA AND THE CARIBBEAN,LATIN AMERICA AND THE CARIBBEAN,Sub-Region,2019,651.036,651036000
68,CENTRAL AMERICA,CENTRAL AMERICA,Sub-Region,2019,178.611,178611000


In [11]:
df_sub_region_population.shape

(24, 6)

In [12]:
# get a list of indices
sub_region_index = pd.Series(df_sub_region_population.index.values.tolist()) 
#create a a list of repeating index values based on range between list items
rep_items = sub_region_index.diff()

In [13]:
df_sub_region_population_copy = df_sub_region_population.copy().reset_index(drop=True)

In [14]:
df_sub_region_population_copy['reps'] = rep_items
df_sub_region_population_copy['reps'] = df_sub_region_population_copy['reps'].shift(periods = -1, fill_value = 18.0)

In [15]:
df_sub_region_population_copy

,FIPS,Name,Type,TimeFrame,Data (M),Population,reps
0,WORLD,WORLD,World,2019,7772.850,7772850000,1.0
1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000,1.0
2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000,8.0
3,WESTERN AFRICA,WESTERN AFRICA,Sub-Region,2019,401.115,401115000,17.0
4,EASTERN AFRICA,EASTERN AFRICA,Sub-Region,2019,444.970,444970000,21.0
5,MIDDLE AFRICA,MIDDLE AFRICA,Sub-Region,2019,179.757,179757000,10.0
6,SOUTHERN AFRICA,SOUTHERN AFRICA,Sub-Region,2019,67.732,67732000,6.0
7,NORTHERN AMERICA,NORTHERN AMERICA,Sub-Region,2019,368.193,368193000,3.0
8,LATIN AMERICA AND THE CARIBBEAN,LATIN AMERICA AND THE CARIBBEAN,Sub-Region,2019,651.036,651036000,1.0
9,CENTRAL AMERICA,CENTRAL AMERICA,Sub-Region,2019,178.611,178611000,9.0


In [16]:
df_sub_region_population_copy = df_sub_region_population_copy.drop([0,1])

In [17]:
df_sub_region_population_copy

,FIPS,Name,Type,TimeFrame,Data (M),Population,reps
2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000,8.0
3,WESTERN AFRICA,WESTERN AFRICA,Sub-Region,2019,401.115,401115000,17.0
4,EASTERN AFRICA,EASTERN AFRICA,Sub-Region,2019,444.970,444970000,21.0
5,MIDDLE AFRICA,MIDDLE AFRICA,Sub-Region,2019,179.757,179757000,10.0
6,SOUTHERN AFRICA,SOUTHERN AFRICA,Sub-Region,2019,67.732,67732000,6.0
7,NORTHERN AMERICA,NORTHERN AMERICA,Sub-Region,2019,368.193,368193000,3.0
8,LATIN AMERICA AND THE CARIBBEAN,LATIN AMERICA AND THE CARIBBEAN,Sub-Region,2019,651.036,651036000,1.0
9,CENTRAL AMERICA,CENTRAL AMERICA,Sub-Region,2019,178.611,178611000,9.0
10,CARIBBEAN,CARIBBEAN,Sub-Region,2019,43.233,43233000,18.0
11,SOUTH AMERICA,SOUTH AMERICA,Sub-Region,2019,429.191,429191000,14.0


In [18]:
sub_reg_reps = df_sub_region_population_copy[['Name', 'reps']]
sub_reg_reps.loc[:, 'reps'] = sub_reg_reps['reps'].apply(lambda x: x - 1).astype(int)
sub_reg_reps

/home/laurawood3/miniconda3/envs/512env/lib/python3.6/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,Name,reps
2,NORTHERN AFRICA,7
3,WESTERN AFRICA,16
4,EASTERN AFRICA,20
5,MIDDLE AFRICA,9
6,SOUTHERN AFRICA,5
7,NORTHERN AMERICA,2
8,LATIN AMERICA AND THE CARIBBEAN,0
9,CENTRAL AMERICA,8
10,CARIBBEAN,17
11,SOUTH AMERICA,13


In [19]:
repeating = sub_reg_reps.loc[sub_reg_reps.index.repeat(sub_reg_reps.reps)]
repeating.shape

(210, 2)

In [20]:
repeating_series = repeating['Name'].squeeze().reset_index(drop=True)
repeating_series

0      NORTHERN AFRICA
1      NORTHERN AFRICA
2      NORTHERN AFRICA
3      NORTHERN AFRICA
4      NORTHERN AFRICA
            ...       
205            OCEANIA
206            OCEANIA
207            OCEANIA
208            OCEANIA
209            OCEANIA
Name: Name, Length: 210, dtype: object

In [21]:
#This code results in erroneously removing the Channel Islands from our analysis
#df_population['Type'].value_counts()
#df_population = df_population[~df_population.Type.str.contains('World')].reset_index(drop=True)
#df_population = df_population[~df_population.Type.str.contains('Sub-Region')].reset_index(drop=True)

In [22]:
df_population = df_population[~df_population['Name'].str.isupper().fillna(False)].reset_index(drop=True)
df_population

,FIPS,Name,Type,TimeFrame,Data (M),Population
0,DZ,Algeria,Country,2019,44.357,44357000
1,EG,Egypt,Country,2019,100.803,100803000
2,LY,Libya,Country,2019,6.891,6891000
3,MA,Morocco,Country,2019,35.952,35952000
4,SD,Sudan,Country,2019,43.849,43849000
...,...,...,...,...,...,...
205,WS,Samoa,Country,2019,0.200,200000
206,SB,Solomon Islands,Country,2019,0.715,715000
207,TO,Tonga,Country,2019,0.099,99000
208,TV,Tuvalu,Country,2019,0.010,10000


In [23]:
df_population.shape

(210, 6)

In [24]:
#add column to df_population that notes the sub-region
df_population['Sub_Region'] = repeating_series

In [25]:
df_population

,FIPS,Name,Type,TimeFrame,Data (M),Population,Sub_Region
0,DZ,Algeria,Country,2019,44.357,44357000,NORTHERN AFRICA
1,EG,Egypt,Country,2019,100.803,100803000,NORTHERN AFRICA
2,LY,Libya,Country,2019,6.891,6891000,NORTHERN AFRICA
3,MA,Morocco,Country,2019,35.952,35952000,NORTHERN AFRICA
4,SD,Sudan,Country,2019,43.849,43849000,NORTHERN AFRICA
...,...,...,...,...,...,...,...
205,WS,Samoa,Country,2019,0.200,200000,OCEANIA
206,SB,Solomon Islands,Country,2019,0.715,715000,OCEANIA
207,TO,Tonga,Country,2019,0.099,99000,OCEANIA
208,TV,Tuvalu,Country,2019,0.010,10000,OCEANIA


In [26]:
df_population.to_csv(r'after_sort.csv', index = False, header = True)

## Step 3: Getting Article Quality Predictions

We need to get the predicted quality scores for each article in the Wikipedia dataset using a machine learning system called ORES that provides estimates of Wikipedia article quality. The article quality estimates (from best to worst) are:

1.	FA - Featured article
2.	GA - Good article
3.	B - B-class article
4.	C - C-class article
5.	Start - Start-class article
6.	Stub - Stub-class article

These were learned based on articles in Wikipedia that were peer-reviewed using the Wikipedia content assessment procedures. These quality classes are a sub-set of quality assessment categories developed by Wikipedia editors. ORES will assign one of these 6 categories to any rev_id we send it.


In order to get article predictions for each article in the Wikipedia dataset, we first need to read page_data.csv into Python, and then read through the dataset line by line, using the value of the rev_id column to make an API query.

In [27]:
from ores import api

In [28]:
headers = {
    'User-Agent': 'https://github.com/laurathriftwood',
    'From': 'lwood3@uw.edu'
}

In [29]:
endpoint = 'https://ores.wikimedia.org/v3/scores/enwiki/?models=articlequality&revids={rev_id}'

We start by extracting a list of `rev_id`'s from our `df_politicians` dataframe for which we want associated ORES scores. Since the full list is quite long, I've left some test batches commented out just in case. The smaller batch includes a `rev_id` for which there is no associate score so we can verify our error handling methodology.

In [30]:
#full batch
rev_list = df_politicians['rev_id']

#test smaller batch
#rev_list = [502721672, 516633096, 521986779]
    
#test larger batch
#rev_list = df_politicians['rev_id'][0:50]

In [31]:
ores_session = api.Session("https://ores.wikimedia.org", "DATA 512 Class project lwood3@uw.edu")

results = ores_session.score("enwiki", ["articlequality"], rev_list)

predictions = []

In [32]:
for score in results:
    while True:
        #attempt to retrieve the prediction for the rev_id and add it to a Series
        try: 
            predictions.append(score["articlequality"]["score"]["prediction"])
            break
            
        #if no prediction is available, we will see an error in the score
        #we note this error with a "No_Score" string entry
        except KeyError:
            predictions.append(str("No_Score"))
            break

Now that we have a list of rev_ids and a list of associated predictions, we merge our results into a single dataframe and verify that all rev_ids were processed by comparing the resulting shape against the length of the original rev_list. 

In [33]:
data = {'rev_id':rev_list, 'prediction':predictions}
#print(data)
predictions_df = pd.DataFrame(data)
print(predictions_df.shape)

(46701, 2)


Now we merge our predictions with the original politicians dataframe using the rev_ids.

In [34]:
merged_df = df_politicians.merge(predictions_df, how = 'inner', on = ['rev_id', 'rev_id'])
merged_df.head()

,page,country,rev_id,prediction
0,Bir I of Kanem,Chad,355319463,Stub
1,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188,Stub
2,Yos Por,Cambodia,393822005,Stub
3,Julius Gregr,Czech Republic,395521877,Stub
4,Edvard Gregr,Czech Republic,395526568,Stub


We check the shape of the merged dataframe to ensure we have not lost any rows of (missing/unmatched) data in the process.

In [35]:
print(merged_df.shape)
merged_df.head()

(46701, 4)


,page,country,rev_id,prediction
0,Bir I of Kanem,Chad,355319463,Stub
1,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188,Stub
2,Yos Por,Cambodia,393822005,Stub
3,Julius Gregr,Czech Republic,395521877,Stub
4,Edvard Gregr,Czech Republic,395526568,Stub


Let's see how many articles without associated scores/predictions.

In [36]:
merged_df.loc[merged_df.prediction == 'No_Score', 'prediction'].count()

276

We see that there are 276 rev_ids that do not have a prediction. We will extract those rows from our working dataframe and store them in a separate output file for our records.

In [37]:
df_politicians_no_score = merged_df[merged_df['prediction'] == 'No_Score']
print(df_politicians_no_score.shape)
df_politicians_no_score.to_csv(r'output/wp_wpds_politicians_no_score.csv', index = False, header = True)

(276, 4)


In [38]:
#drop rows that have a No_Score prediction value and check the shape to ensure 276 rows were dropped
merged_df = merged_df[~merged_df.prediction.str.contains('No_Score')].reset_index(drop=True)
merged_df.shape

(46425, 4)

In [39]:
df_population.shape

(210, 7)

In [40]:
merged_df.shape

(46425, 4)

## Combining the Datasets

We now need to merge our two datasets - the Wikipedia data in `merged_df` and the population data in `df_population` on their respective __country__ and __Name__ fields. Since we want maintain a record of the subset that does not have matching data, we will use an outer join to retain those rows.

In [41]:
all_data_df = merged_df.merge(df_population, how = 'outer', left_on='country', right_on='Name')
print(all_data_df.shape)
all_data_df

(46452, 11)


,page,country,rev_id,prediction,FIPS,Name,Type,TimeFrame,Data (M),Population,Sub_Region
0,Bir I of Kanem,Chad,355319463.0,Stub,TD,Chad,Country,2019.0,16.877,16877000.0,MIDDLE AFRICA
1,Abdullah II of Kanem,Chad,498683267.0,Stub,TD,Chad,Country,2019.0,16.877,16877000.0,MIDDLE AFRICA
2,Salmama II of Kanem,Chad,565745353.0,Stub,TD,Chad,Country,2019.0,16.877,16877000.0,MIDDLE AFRICA
3,Kuri I of Kanem,Chad,565745365.0,Stub,TD,Chad,Country,2019.0,16.877,16877000.0,MIDDLE AFRICA
4,Mohammed I of Kanem,Chad,565745375.0,Stub,TD,Chad,Country,2019.0,16.877,16877000.0,MIDDLE AFRICA
...,...,...,...,...,...,...,...,...,...,...,...
46447,NaN,NaN,NaN,NaN,PF,French Polynesia,Country,2019.0,0.280,280000.0,OCEANIA
46448,NaN,NaN,NaN,NaN,GU,Guam,Country,2019.0,0.175,175000.0,OCEANIA
46449,NaN,NaN,NaN,NaN,NC,New Caledonia,Country,2019.0,0.295,295000.0,OCEANIA
46450,NaN,NaN,NaN,NaN,PW,Palau,Country,2019.0,0.018,18000.0,OCEANIA


We want to extract rows in the __country__ or __Name__ columns where our data doesn't match (contains NaN values) and export it to a .csv file for our records. We are only interested in removing rows with NaN values in the country/Name columns, but as there are NaN values in the FIPS column, we need to be specific in our column operations

In [42]:
df_no_match = all_data_df[(all_data_df['country'].isnull()) | (all_data_df['Name'].isnull())] #1884 rows
df_no_match.to_csv(r'output/wp_wpds_countries-no_match.csv', index = False, header = True)
print(df_no_match.shape)
df_no_match

(1884, 11)


,page,country,rev_id,prediction,FIPS,Name,Type,TimeFrame,Data (M),Population,Sub_Region
488,Julius Gregr,Czech Republic,395521877.0,Stub,NaN,NaN,NaN,NaN,NaN,NaN,NaN
489,Edvard Gregr,Czech Republic,395526568.0,Stub,NaN,NaN,NaN,NaN,NaN,NaN,NaN
490,Miroslav Poche,Czech Republic,672862914.0,Stub,NaN,NaN,NaN,NaN,NaN,NaN,NaN
491,Vojtěch Mynář,Czech Republic,673008587.0,Stub,NaN,NaN,NaN,NaN,NaN,NaN,NaN
492,Jan Malypetr,Czech Republic,704424304.0,Stub,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
46447,NaN,NaN,NaN,NaN,PF,French Polynesia,Country,2019.0,0.280,280000.0,OCEANIA
46448,NaN,NaN,NaN,NaN,GU,Guam,Country,2019.0,0.175,175000.0,OCEANIA
46449,NaN,NaN,NaN,NaN,NC,New Caledonia,Country,2019.0,0.295,295000.0,OCEANIA
46450,NaN,NaN,NaN,NaN,PW,Palau,Country,2019.0,0.018,18000.0,OCEANIA


 We will also drop these rows from our working dataframe and check the shape before and after to ensure it matched the number of rows we identified in the previous step.

In [43]:
print(all_data_df.shape)
all_data_df = all_data_df.dropna(subset = ['country', 'Name']).reset_index(drop=True)
print(all_data_df.shape)

(46452, 11)
(44568, 11)


Let's clean up our dataset to match the schema in the assignment instructions by dropping unnecessary columns, renaming the column headers, and reordering the columns. Then we can export our final dataset to a .csv file.

In [44]:
#convert rev_id and Population to integer
all_data_df = all_data_df.astype({"rev_id": int, "Population": int})

In [45]:
all_data_df

,page,country,rev_id,prediction,FIPS,Name,Type,TimeFrame,Data (M),Population,Sub_Region
0,Bir I of Kanem,Chad,355319463,Stub,TD,Chad,Country,2019.0,16.877,16877000,MIDDLE AFRICA
1,Abdullah II of Kanem,Chad,498683267,Stub,TD,Chad,Country,2019.0,16.877,16877000,MIDDLE AFRICA
2,Salmama II of Kanem,Chad,565745353,Stub,TD,Chad,Country,2019.0,16.877,16877000,MIDDLE AFRICA
3,Kuri I of Kanem,Chad,565745365,Stub,TD,Chad,Country,2019.0,16.877,16877000,MIDDLE AFRICA
4,Mohammed I of Kanem,Chad,565745375,Stub,TD,Chad,Country,2019.0,16.877,16877000,MIDDLE AFRICA
...,...,...,...,...,...,...,...,...,...,...,...
44563,Rita Sinon,Seychelles,800323154,Stub,SC,Seychelles,Country,2019.0,0.098,98000,EASTERN AFRICA
44564,Sylvette Frichot,Seychelles,800323798,Stub,SC,Seychelles,Country,2019.0,0.098,98000,EASTERN AFRICA
44565,May De Silva,Seychelles,800969960,Start,SC,Seychelles,Country,2019.0,0.098,98000,EASTERN AFRICA
44566,Vincent Meriton,Seychelles,802051093,Stub,SC,Seychelles,Country,2019.0,0.098,98000,EASTERN AFRICA


In [46]:
#convert rev_id and Population to integer
all_data_df = all_data_df.astype({"rev_id": int, "Population": int})

#drop unnecessary columns
all_data_df = all_data_df.drop(all_data_df.columns[[4, 5, 6, 7, 8]], axis=1)
all_data_df

#rename columns
all_data_df = all_data_df.rename(columns={'page': 'article_name', 
                                          'rev_id': 'revision_id', 
                                          'prediction': 'article_quality_est', 
                                          'Population': 'population',
                                         'Sub_Region': 'subregion'})

#reorder columns
all_data_df = all_data_df[['country', 'subregion', 'article_name', 'revision_id', 'article_quality_est', 'population']]

In [47]:
all_data_df

,country,subregion,article_name,revision_id,article_quality_est,population
0,Chad,MIDDLE AFRICA,Bir I of Kanem,355319463,Stub,16877000
1,Chad,MIDDLE AFRICA,Abdullah II of Kanem,498683267,Stub,16877000
2,Chad,MIDDLE AFRICA,Salmama II of Kanem,565745353,Stub,16877000
3,Chad,MIDDLE AFRICA,Kuri I of Kanem,565745365,Stub,16877000
4,Chad,MIDDLE AFRICA,Mohammed I of Kanem,565745375,Stub,16877000
...,...,...,...,...,...,...
44563,Seychelles,EASTERN AFRICA,Rita Sinon,800323154,Stub,98000
44564,Seychelles,EASTERN AFRICA,Sylvette Frichot,800323798,Stub,98000
44565,Seychelles,EASTERN AFRICA,May De Silva,800969960,Start,98000
44566,Seychelles,EASTERN AFRICA,Vincent Meriton,802051093,Stub,98000


In [48]:
all_data_df

,country,subregion,article_name,revision_id,article_quality_est,population
0,Chad,MIDDLE AFRICA,Bir I of Kanem,355319463,Stub,16877000
1,Chad,MIDDLE AFRICA,Abdullah II of Kanem,498683267,Stub,16877000
2,Chad,MIDDLE AFRICA,Salmama II of Kanem,565745353,Stub,16877000
3,Chad,MIDDLE AFRICA,Kuri I of Kanem,565745365,Stub,16877000
4,Chad,MIDDLE AFRICA,Mohammed I of Kanem,565745375,Stub,16877000
...,...,...,...,...,...,...
44563,Seychelles,EASTERN AFRICA,Rita Sinon,800323154,Stub,98000
44564,Seychelles,EASTERN AFRICA,Sylvette Frichot,800323798,Stub,98000
44565,Seychelles,EASTERN AFRICA,May De Silva,800969960,Start,98000
44566,Seychelles,EASTERN AFRICA,Vincent Meriton,802051093,Stub,98000


In [49]:
#make a copy to export that drops the subregion
all_data_df_export = all_data_df.drop(all_data_df.columns[[1]], axis=1)
all_data_df_export

,country,article_name,revision_id,article_quality_est,population
0,Chad,Bir I of Kanem,355319463,Stub,16877000
1,Chad,Abdullah II of Kanem,498683267,Stub,16877000
2,Chad,Salmama II of Kanem,565745353,Stub,16877000
3,Chad,Kuri I of Kanem,565745365,Stub,16877000
4,Chad,Mohammed I of Kanem,565745375,Stub,16877000
...,...,...,...,...,...
44563,Seychelles,Rita Sinon,800323154,Stub,98000
44564,Seychelles,Sylvette Frichot,800323798,Stub,98000
44565,Seychelles,May De Silva,800969960,Start,98000
44566,Seychelles,Vincent Meriton,802051093,Stub,98000


In [50]:
#make a copy to export that drops the subregion
all_data_df_export = all_data_df.drop(all_data_df.columns[[1]], axis=1)

#output to file as a .csv
all_data_df_export.to_csv(r'output/wp_wpds_politicians_by_country.csv', index = False, header = True)

Let's take a look at our final dataset.

In [51]:
all_data_df

,country,subregion,article_name,revision_id,article_quality_est,population
0,Chad,MIDDLE AFRICA,Bir I of Kanem,355319463,Stub,16877000
1,Chad,MIDDLE AFRICA,Abdullah II of Kanem,498683267,Stub,16877000
2,Chad,MIDDLE AFRICA,Salmama II of Kanem,565745353,Stub,16877000
3,Chad,MIDDLE AFRICA,Kuri I of Kanem,565745365,Stub,16877000
4,Chad,MIDDLE AFRICA,Mohammed I of Kanem,565745375,Stub,16877000
...,...,...,...,...,...,...
44563,Seychelles,EASTERN AFRICA,Rita Sinon,800323154,Stub,98000
44564,Seychelles,EASTERN AFRICA,Sylvette Frichot,800323798,Stub,98000
44565,Seychelles,EASTERN AFRICA,May De Silva,800969960,Start,98000
44566,Seychelles,EASTERN AFRICA,Vincent Meriton,802051093,Stub,98000


## Step 4: Analysis

We'd like to calculate the proportion (as a percentage) of artcicles-per-population and high-quality articles for each country AND for each geographic region. 

"High quality" here is defined as having ORES quality prediction scores as either "FA" - Featured Article or "GA" - Good Article. As such, we add a column that converts the __article_quality_est__ predictions into binary indicators, with $1$ representing FA or GA scores and $0$ representing all other scores.

We then need to create a dataframe that groups our data by __country__ and by __high_quality__. 

To generate the six tables requested in Step 5, we will need a table with the following columns, with one row for each country in our dataset:
- country
- geographic region
- population
- total number of articles
- total number of high quality articles
- coverage (number of articles / population)
- relative quality (number of high quality articles / number of articles)

In [52]:
all_data_df['high_quality'] = np.where(all_data_df.article_quality_est.str.contains('GA' or 'FA'), 1, 0)
all_data_df

,country,subregion,article_name,revision_id,article_quality_est,population,high_quality
0,Chad,MIDDLE AFRICA,Bir I of Kanem,355319463,Stub,16877000,0
1,Chad,MIDDLE AFRICA,Abdullah II of Kanem,498683267,Stub,16877000,0
2,Chad,MIDDLE AFRICA,Salmama II of Kanem,565745353,Stub,16877000,0
3,Chad,MIDDLE AFRICA,Kuri I of Kanem,565745365,Stub,16877000,0
4,Chad,MIDDLE AFRICA,Mohammed I of Kanem,565745375,Stub,16877000,0
...,...,...,...,...,...,...,...
44563,Seychelles,EASTERN AFRICA,Rita Sinon,800323154,Stub,98000,0
44564,Seychelles,EASTERN AFRICA,Sylvette Frichot,800323798,Stub,98000,0
44565,Seychelles,EASTERN AFRICA,May De Silva,800969960,Start,98000,0
44566,Seychelles,EASTERN AFRICA,Vincent Meriton,802051093,Stub,98000,0


In [53]:
#create new dataframe that gets a count of high_quality articles per country
all_data_df_quality = all_data_df[(all_data_df['high_quality'] == 1)]
all_data_df_quality

,country,subregion,article_name,revision_id,article_quality_est,population,high_quality
82,Chad,MIDDLE AFRICA,Hissène Habré,803166806,GA,16877000,1
199,Palestinian Territory,WESTERN ASIA,Abdullah Rimawi,788953220,GA,5008000,1
204,Palestinian Territory,WESTERN ASIA,Khalida Jarrar,791881528,GA,5008000,1
218,Palestinian Territory,WESTERN ASIA,Ahmed Yassin,797122322,GA,5008000,1
225,Palestinian Territory,WESTERN ASIA,Marwan Barghouti,798913975,GA,5008000,1
...,...,...,...,...,...,...,...
44292,Saudi Arabia,WESTERN ASIA,Mohammad bin Salman,807463170,GA,35041000,1
44293,Saudi Arabia,WESTERN ASIA,Fahd of Saudi Arabia,807483153,GA,35041000,1
44321,Trinidad and Tobago,CARIBBEAN,Jack Warner (football executive),805253461,GA,1369000,1
44368,Dominica,CARIBBEAN,Eugenia Charles,802175384,GA,72000,1


In [54]:
#country, region, and population count table
summary_df = all_data_df.groupby(['country', 'subregion', 'population']).sum('high_quality').reset_index()
summary_df

,country,subregion,population,revision_id,high_quality
0,Afghanistan,SOUTH ASIA,38928000,247043956038,12
1,Albania,SOUTHERN EUROPE,2838000,357171119064,3
2,Algeria,NORTHERN AFRICA,44357000,90374847695,2
3,Andorra,SOUTHERN EUROPE,82000,26179556617,0
4,Angola,MIDDLE AFRICA,32522000,80804303350,0
...,...,...,...,...,...
178,Venezuela,SOUTH AMERICA,28645000,100771979914,3
179,Vietnam,SOUTHEAST ASIA,96209000,145278362558,6
180,Yemen,WESTERN ASIA,29826000,89361103584,2
181,Zambia,EASTERN AFRICA,18384000,19659512534,0


In [55]:
summary_df = summary_df.drop(columns = ['revision_id'])
summary_df

,country,subregion,population,high_quality
0,Afghanistan,SOUTH ASIA,38928000,12
1,Albania,SOUTHERN EUROPE,2838000,3
2,Algeria,NORTHERN AFRICA,44357000,2
3,Andorra,SOUTHERN EUROPE,82000,0
4,Angola,MIDDLE AFRICA,32522000,0
...,...,...,...,...
178,Venezuela,SOUTH AMERICA,28645000,3
179,Vietnam,SOUTHEAST ASIA,96209000,6
180,Yemen,WESTERN ASIA,29826000,2
181,Zambia,EASTERN AFRICA,18384000,0


In [56]:
#total number of articles by country
article_counts = all_data_df[['country', 'article_name']].groupby("country").count().astype(int).reset_index()
article_counts

,country,article_name
0,Afghanistan,319
1,Albania,456
2,Algeria,116
3,Andorra,34
4,Angola,106
...,...,...
178,Venezuela,130
179,Vietnam,187
180,Yemen,116
181,Zambia,25


In [57]:
#merge the above data into a new table that includes population
combined = summary_df.merge(article_counts, how = 'left', on='country')

#rename article_name to total_articles
combined.rename(columns={'article_name': 'total_articles'}, inplace=True)  
combined

,country,subregion,population,high_quality,total_articles
0,Afghanistan,SOUTH ASIA,38928000,12,319
1,Albania,SOUTHERN EUROPE,2838000,3,456
2,Algeria,NORTHERN AFRICA,44357000,2,116
3,Andorra,SOUTHERN EUROPE,82000,0,34
4,Angola,MIDDLE AFRICA,32522000,0,106
...,...,...,...,...,...
178,Venezuela,SOUTH AMERICA,28645000,3,130
179,Vietnam,SOUTHEAST ASIA,96209000,6,187
180,Yemen,WESTERN ASIA,29826000,2,116
181,Zambia,EASTERN AFRICA,18384000,0,25


Let's make a copy of this starter table for our subregion analysis and drop the extra columns. We'll also drop the subregion column from the combined table.

In [58]:
combined_sub = combined.copy()
combined_sub = combined_sub.drop(columns = ['country'])
combined_sub = combined_sub.groupby(['subregion']).sum('high_quality').reset_index()
combined = combined.drop(columns = ['subregion'])

Now that we have our columns with basic counts, let's add columns to calculate:
- __%coverage__ which we define as the proportion of all of a country's articles of high_quality (FA or GA)
- __%relative_quality__ which we define as the proportion of high_quality articles per population

We want to display the actual percentage so we multiply by 100. I've chosen to round to 4 decimal points as the percentages for the __%relative_quality__ values are so small.

We do this for both the combined (by country) table and the combined_sub (by subregion) table.

In [59]:
#coverage (number of high quality articles/population)
combined['%coverage'] = combined.apply(lambda x: round(((x['high_quality']/x['total_articles'])*100), 4), axis=1)

#relative quality (number of high quality articles/total number of articles)
combined['%relative_quality'] = combined.apply(lambda x: round(((x['high_quality']/x['population'])*100), 4), axis=1)

In [60]:
#coverage (number of high quality articles/population)
combined_sub['%coverage'] = combined_sub.apply(lambda x: round(((x['high_quality']/x['total_articles'])*100), 4), axis=1)

#relative quality (number of high quality articles/total number of articles)
combined_sub['%relative_quality'] = combined_sub.apply(lambda x: round(((x['high_quality']/x['population'])*100), 4), axis=1)

In [61]:
#number of high quality articles by country
#quality_article_counts = all_data_df_quality[['country', 'article_name']].groupby('country').count().astype(int).reset_index()
#quality_article_counts

In [62]:
combined

,country,population,high_quality,total_articles,%coverage,%relative_quality
0,Afghanistan,38928000,12,319,3.7618,0.0000
1,Albania,2838000,3,456,0.6579,0.0001
2,Algeria,44357000,2,116,1.7241,0.0000
3,Andorra,82000,0,34,0.0000,0.0000
4,Angola,32522000,0,106,0.0000,0.0000
...,...,...,...,...,...,...
178,Venezuela,28645000,3,130,2.3077,0.0000
179,Vietnam,96209000,6,187,3.2086,0.0000
180,Yemen,29826000,2,116,1.7241,0.0000
181,Zambia,18384000,0,25,0.0000,0.0000


In [63]:
combined_sub

,subregion,population,high_quality,total_articles,%coverage,%relative_quality
0,CARIBBEAN,39056000,11,695,1.5827,0.0000
1,CENTRAL AMERICA,162267000,16,1543,1.0369,0.0000
2,CENTRAL ASIA,74960000,6,245,2.4490,0.0000
3,EAST ASIA,1632883000,59,2473,2.3858,0.0000
4,EASTERN AFRICA,443825000,28,2502,1.1191,0.0000
5,EASTERN EUROPE,281186000,68,3732,1.8221,0.0000
6,MIDDLE AFRICA,90189000,11,665,1.6541,0.0000
7,NORTHERN AFRICA,243748000,11,899,1.2236,0.0000
8,NORTHERN AMERICA,368068000,73,1901,3.8401,0.0000
9,NORTHERN EUROPE,105680000,75,3763,1.9931,0.0001


## Step 5: Results

Now that we have the table we need to produce the six results tables requested in the assignment.

### Table 1
#### Top 10 countries by coverage: 10 highest-ranked countries in terms of number of politician articles as a proportion of country population.

We start by sorting our table on the __%coverage__ column and producing the top 10 rows.

In [64]:
combined.sort_values(['%coverage'], axis=0, ascending=False, inplace = True, kind='quicksort', ignore_index=True)
combined.head(10)

,country,population,high_quality,total_articles,%coverage,%relative_quality
0,"Korea, North",25779000,7,36,19.4444,0.0000
1,Saudi Arabia,35041000,13,117,11.1111,0.0000
2,Central African Republic,4830000,6,66,9.0909,0.0001
3,Mauritania,4650000,4,48,8.3333,0.0001
4,Dominica,72000,1,12,8.3333,0.0014
5,Tuvalu,10000,4,54,7.4074,0.0400
6,Singapore,5769000,5,68,7.3529,0.0001
7,Uzbekistan,34174000,2,28,7.1429,0.0000
8,Bhutan,730000,2,33,6.0606,0.0003
9,Guatemala,18066000,5,83,6.0241,0.0000


### Table 2
#### Bottom 10 countries by coverage: 10 lowest-ranked countries in terms of number of politician articles as a proportion of country population

Since we already have our table sorted, we instead output 10 entries from the tail of our sorted dataframe.

In [65]:
combined.tail(10)

,country,population,high_quality,total_articles,%coverage,%relative_quality
173,Nauru,11000,0,52,0.0,0.0
174,Nicaragua,6596000,0,114,0.0,0.0
175,Peru,32824000,0,350,0.0,0.0
176,Bolivia,11633000,0,183,0.0,0.0
177,Belize,419000,0,16,0.0,0.0
178,Rwanda,12952000,0,104,0.0,0.0
179,San Marino,34000,0,81,0.0,0.0
180,Sao Tome and Principe,210000,0,21,0.0,0.0
181,Belarus,9375000,0,72,0.0,0.0
182,Mozambique,31166000,0,58,0.0,0.0


### Table 3
#### Top 10 countries by relative quality: 10 highest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality.

We simply need to resort our table using the __%relative_quality__ column for this table, and output the top 10 entries.

In [66]:
combined.sort_values(['%relative_quality'], axis=0, ascending=False, inplace = True, kind='quicksort', ignore_index=True)
combined.head(10)

,country,population,high_quality,total_articles,%coverage,%relative_quality
0,Tuvalu,10000,4,54,7.4074,0.0400
1,Dominica,72000,1,12,8.3333,0.0014
2,Vanuatu,321000,3,58,5.1724,0.0009
3,Iceland,368000,2,201,0.9950,0.0005
4,Martinique,356000,1,34,2.9412,0.0003
5,Ireland,5003000,17,373,4.5576,0.0003
6,Montenegro,622000,2,72,2.7778,0.0003
7,Bhutan,730000,2,33,6.0606,0.0003
8,New Zealand,4987000,11,783,1.4049,0.0002
9,Maldives,541000,1,83,1.2048,0.0002


### Table 4
#### Bottom 10 countries by relative quality: 10 lowest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality

Again, as we already have our table sorted on this value from the previous table, we simply need to output 10 items from the tail.

In [67]:
combined.tail(10)

,country,population,high_quality,total_articles,%coverage,%relative_quality
173,Togo,8279000,1,65,1.5385,0.0
174,Spain,47635000,13,871,1.4925,0.0
175,Burundi,11866000,1,75,1.3333,0.0
176,Iran,84150000,12,810,1.4815,0.0
177,Pakistan,220940000,15,1019,1.4720,0.0
178,France,64940000,24,1672,1.4354,0.0
179,Cambodia,15497000,3,213,1.4085,0.0
180,Serbia,6957000,3,215,1.3953,0.0
181,Japan,125967000,6,437,1.3730,0.0
182,Mozambique,31166000,0,58,0.0000,0.0


### Table 5
#### Geographic regions by coverage: Ranking of geographic regions (in descending order) in terms of the total count of politician articles from countries in each region as a proportion of total regional population

For this table, we simply sort our subregion table on the __%

In [68]:
combined_sub.sort_values(['%relative_quality'], axis=0, ascending=False, inplace = True, kind='quicksort', ignore_index=True)
combined_sub

,subregion,population,high_quality,total_articles,%coverage,%relative_quality
0,NORTHERN EUROPE,105680000,75,3763,1.9931,0.0001
1,OCEANIA,42031000,49,3126,1.5675,0.0001
2,WESTERN ASIA,272499000,70,2563,2.7312,0.0000
3,WESTERN AFRICA,374933000,31,2139,1.4493,0.0000
4,SOUTHERN EUROPE,151136000,37,3710,0.9973,0.0000
5,SOUTHERN AFRICA,66628000,7,634,1.1041,0.0000
6,SOUTHEAST ASIA,660056000,54,2020,2.6733,0.0000
7,SOUTH ASIA,1967131000,68,4366,1.5575,0.0000
8,SOUTH AMERICA,429188000,29,3032,0.9565,0.0000
9,CARIBBEAN,39056000,11,695,1.5827,0.0000


### Table 6
#### Geographic regions by coverage: Ranking of geographic regions (in descending order) in terms of the relative proportion of politician articles from countries in each region that are of GA and FA-quality

In [69]:
combined_sub.sort_values(['%coverage'], axis=0, ascending=False, inplace = True, kind='quicksort', ignore_index=True)
combined_sub

,subregion,population,high_quality,total_articles,%coverage,%relative_quality
0,NORTHERN AMERICA,368068000,73,1901,3.8401,0.0000
1,WESTERN ASIA,272499000,70,2563,2.7312,0.0000
2,SOUTHEAST ASIA,660056000,54,2020,2.6733,0.0000
3,CENTRAL ASIA,74960000,6,245,2.4490,0.0000
4,EAST ASIA,1632883000,59,2473,2.3858,0.0000
5,NORTHERN EUROPE,105680000,75,3763,1.9931,0.0001
6,EASTERN EUROPE,281186000,68,3732,1.8221,0.0000
7,MIDDLE AFRICA,90189000,11,665,1.6541,0.0000
8,CARIBBEAN,39056000,11,695,1.5827,0.0000
9,OCEANIA,42031000,49,3126,1.5675,0.0001


## Writeup: Reflections and Implications